In [ ]:
import pandas as pd

# Carregando o DataFrame tratado
df = pd.read_csv('TelecomX_data_tratados.csv')

print("Colunas originais antes da remoção:")
print(df.columns)
print("\nPrimeiras 5 linhas do DataFrame original:")
print(df.head())

# Identificando e eliminando colunas que não agregam valor preditivo
# 'ID_Cliente' é um identificador único e geralmente não é útil para prever churn.
# 'Unnamed: 0' é uma coluna de índice que pode ter sido gerada ao salvar o CSV, e também não é útil.
colunas_para_eliminar = ['ID_Cliente', 'Unnamed: 0'] 

# Verificando se as colunas existem antes de tentar eliminá-las para evitar erros
colunas_existentes_para_eliminar = [col for col in colunas_para_eliminar if col in df.columns]

if colunas_existentes_para_eliminar:
    df = df.drop(columns=colunas_existentes_para_eliminar)
    print(f"\nColunas eliminadas: {colunas_existentes_para_eliminar}")
else:
    print("\nNenhuma das colunas especificadas para eliminação foi encontrada.")

print("\nColunas após a eliminação:")
print(df.columns)
print("\nPrimeiras 5 linhas do DataFrame após a eliminação:")
print(df.head())

Colunas originais antes da remoção:
Index(['ID_Cliente', 'Cancelou', 'Genero', 'Idoso', 'Possui_Conjuge',
       'Possui_Dependentes', 'Meses_Permanencia', 'Servico_Telefone',
       'Multiplas_Linhas', 'Tipo_Internet', 'OnlineSecurity', 'Backup_Online',
       'Protecao_Dispositivo', 'Suporte_Tecnico', 'Streaming_TV',
       'Streaming_Filmes', 'Tipo_Contrato', 'Fatura_Digital',
       'Metodo_Pagamento', 'Cobranca_Mensal', 'Cobranca_Total'],
      dtype='object')

Primeiras 5 linhas do DataFrame original:
   ID_Cliente Cancelou     Genero  Idoso Possui_Conjuge Possui_Dependentes  \
0  0002-ORFBO      Não   Feminino      0            Sim                Sim   
1  0003-MKNFE      Não  Masculino      0            Não                Não   
2  0004-TLHLJ      Sim  Masculino      0            Não                Não   
3  0011-IGKFF      Sim  Masculino      1            Sim                Não   
4  0013-EXCHZ      Sim   Feminino      1            Sim                Não   

   Meses_Permanenc